In [42]:

import scipy.io
import numpy as np
import pandas as pd
import seaborn as sns
import tensorly as tl
import math 
import matplotlib.pyplot as plt
import gzip 

sns.set_theme()

#### Algorithm 1

In [2]:
def my_HOSVD(T):

    U1, _, _ = np.linalg.svd(tl.unfold(T, 0), full_matrices=False) 
    U2, _, _ = np.linalg.svd(tl.unfold(T, 1), full_matrices=False) 
    U3, _, _ = np.linalg.svd(tl.unfold(T, 2), full_matrices=False)

    S = tl.tenalg.multi_mode_dot(T, [np.transpose(U1),np.transpose(U2),np.transpose(U3)], modes = [0,1,2], transpose=False)
    
    return(S, U1, U2)

In [32]:
def alg1_train(A_train, D_train,  k):
    
    tensor = [None] * k
    
    for i in range(k):

        T = tl.vec_to_tensor(A_train[:, np.where(D_train == i)[1]], (int(math.sqrt(A_train.shape[0])),int(math.sqrt(A_train.shape[0])),A_train[:, np.where(D_train == i)[1]].shape[1]))
        S, U1, U2 = my_HOSVD(T)
    
        tensor[i] = np.zeros((int(math.sqrt(A_train.shape[0])),int(math.sqrt(A_train.shape[0])), S.shape[2]))

        for j in range(S.shape[2]):
            pom_1 = multiplication(tl.vec_to_tensor(S[:,:,j], (int(math.sqrt(A_train.shape[0])),int(math.sqrt(A_train.shape[0])),1)), U1, 0)
            pom_2 = multiplication(pom_1, U2, 1)
            tensor[i][:, :, j] = np.reshape(pom_2 / np.linalg.norm(pom_2), (int(math.sqrt(A_train.shape[0])),int(math.sqrt(A_train.shape[0]))))
            
    return(tensor)

In [39]:
def alg1_test(A_test, tensor,  k, l):
        
    digits = []

    for i in range(A_test.shape[1]): 
        
        D = np.reshape((A_test[:, i] / np.linalg.norm(A_test[:, i])), (int(math.sqrt(A_train.shape[0])),int(math.sqrt(A_train.shape[0]))))
        
        max_R = np.NINF
        digit = 0 
    
        for j in range(k):
            
            R = 0 

            for z in range(l): 
                R += np.trace(np.matmul(np.transpose(D), tensor[j][:,:,z])) ** 2  
                
            if( R > max_R): 
                max_R = R
                digit = j
            
        digits.append(digit)
        
    return digits

In [40]:

##reading data
mat = scipy.io.loadmat('azip.mat')
A_train = mat['azip'] 
mat = scipy.io.loadmat('dzip.mat')
D_train = mat['dzip'] 

mat = scipy.io.loadmat('atest.mat')
A_test = mat['testzip'] 
mat = scipy.io.loadmat('dtest.mat')
D_test = mat['dtest'] 

k = np.unique(D_train).size
l = 30

In [41]:
tensor = alg1_train(A_train, D_train, k )

predictions = alg1_test(A_test, tensor, k, l)

np.count_nonzero(predictions == D_test[0]) / len(D_test[0])

0.9342301943198804

In [59]:
#plotanje norma

def plot_norm(T):
    
    norm_1 = np.zeros(T.shape[0]) 
    for i in range(T.shape[0]):
        norm_1[i] = np.linalg.norm(T[i,:,:]) ** 2 #optimizirati naknadno: suma dij!

    norm_2 = np.zeros(T.shape[1])   
    for i in range(T.shape[1]):
        norm_2[i] = np.linalg.norm(T[:,i,:]) ** 2

    norm_3 = np.zeros(T.shape[2])   
    for i in range(T.shape[2]):
        norm_3[i] = np.linalg.norm(T[:,:,i]) ** 2

    df_ = pd.DataFrame(data = {'Mode 1':norm_1, 'Mode 2':norm_2}) #opt isto
    df__ = pd.DataFrame(data = {'Mode 3':norm_3})
    df = pd.concat([df_,df__], ignore_index=True, axis=1)

    sns.lineplot(data = df)
    

#### 25.2.2022. by E Algoritam 2 

In [7]:
def multiplication(T, M, mode): 
    descriptor = list(T.shape)
    descriptor[mode] = M.shape[0]
    pom = tl.unfold(T, mode)
    return (tl.fold( M @ pom, mode, descriptor))   

In [3]:
def HOSVD2(T, p, q): 
    U1, _, _ = np.linalg.svd(tl.unfold(T, 0), full_matrices=False)
    U2, _, _ = np.linalg.svd(tl.unfold(T, 1), full_matrices=False)
    F_ = multiplication(T, np.transpose(U1[:, 0:p]), 0)
    F = multiplication(F_, np.transpose(U2[:, 0:q]), 1)
    return F, U1[:, 0:p]

In [4]:
def alg2_train(F, k):
    B = np.zeros((F.shape[0], k, k))
    for mi in range(F.shape[2]):
        U, _, _ = np.linalg.svd(F[:, :, mi], full_matrices=False)
        B[:, :, mi] = U[:, 0:k]
    return(B)

In [5]:
def alg2_test(A_test, B, U_p):
    
    digits = []
    
    for j in range(A_test.shape[1]): 
        
        d = A_test[:, j]
        d = np.transpose(U_p) @ d 
        
        r_min =  700000
        
        for k in range(B.shape[2]): 

            r = np.linalg.norm(d - (B[:,:,k] @ (np.transpose(B[:,:, k]) @ d)))
            
            if(r < r_min):
                r_min = r 
                digit = k
                
        digits.append(digit)

    return digits        
    

In [51]:
def construct_tensor(A_train):
    makszn = (D_train[0] == np.bincount(D_train[0]).argmax()).sum()
    T = np.zeros((A_train.shape[0],makszn,k))

    for i in range(k):    
        for j in range(makszn):       
            if(j >= len(np.where(D_train==i)[1])):
                mjesto = np.random.randint(0, len(np.where(D_train==i)[1])-1)           
                T[:,j,i]=A_train[:, np.where(D_train == i)[1]][:,mjesto]
        
            else:
                T[:,j,i]=A_train[:, np.where(D_train == i)[1]][:,j]
    
    return(T)


In [53]:

mat = scipy.io.loadmat('azip.mat')
A_train = mat['azip'] #matrix 265 x 1707, one column is one digit
mat = scipy.io.loadmat('dzip.mat')
D_train = mat['dzip'] #matrix 1 x 1707, info about A

mat = scipy.io.loadmat('atest.mat')
A_test = mat['testzip'] 
mat = scipy.io.loadmat('dtest.mat')
D_test = mat['dtest'] 

k = np.unique(D_train).size

#pd.Series(D_train[0]).value_counts() #broj uzorka za svaku znamenku

In [54]:
T = construct_tensor(A_train)

F, U_p = HOSVD2(T, 128, 128) 

B = alg2_train(F,k)

predictions = alg2_test(A_test, B, U_p)

np.count_nonzero(predictions == D_test[0]) / len(D_test[0])

0.9282511210762332

In [45]:
##loading data
f = gzip.open("MNIST_train_img.gz", "r")
file_content=f.read()

In [68]:
import numpy as np
import requests, gzip, os, hashlib
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/eleonoradetic/opt/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['f', 'mat']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [69]:
#fetch data
path='/mnt/e/ADITYA/EDUCATION/ML/Jupyter Lab/MNIST/mnist-from-numpy/data'
def fetch(url):
    fp = os.path.join(path, hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp):
        with open(fp, "rb") as f:
            data = f.read()
    else:
        with open(fp, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(gzip.decompress(data), dtype=np.uint8).copy()

X = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28*28))
Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/e/ADITYA/EDUCATION/ML/Jupyter Lab/MNIST/mnist-from-numpy/data/23278f029ff68f1e993776e500ce06b9'